# beautiful soup

In [ ]:
import bs4 as bs
import urllib.request

url = 'https://www.quora.com/search?q=founder&type=profile'


In [ ]:
source = urllib.request.urlopen(url).read()

In [ ]:
soup = bs.BeautifulSoup(source,'lxml')

In [ ]:
print(soup)

# selenium

In [1]:
from selenium.webdriver.common.keys import Keys
import selenium.webdriver


In [34]:
driver = selenium.webdriver.Chrome('/Users/vkwan/Dev/chromedriver')
driver.get("https://www.quora.com/search?q=founder&type=profile")

In [37]:
driver.window_handles

['CDwindow-AB9689B7DF420EDF58027966980EFA7B']

In [39]:
element = driver.find_element_by_id("__w2_wFrV4ViP11_content")
element.text

"By Type\nAll Types\nQuestions\nAnswers\nPosts\nProfiles\nTopics\nBlogs\nSpaces\nProfiles for founder\nClear Filters\n Profile:\nCharlie Cheever, Quora\nIn addition to being a founder at Quora: Palo Alto, Facebook, Amazon, Seattle, Harvard, Shady Side Academy, Pittsburgh… (more)\nProfile:\nMark Ferguson, Founder InvestFourMore, Real estate Investor and Broker at Blue Steel Real Estate\n17 year real estate investor, real estate agent, best-selling author, and car lover. I have flipped more than 155 houses, and bought 20 rentals in my career. I have 19 flip...(more)\nProfile:\nDave Consiglio, Chemistry and Physics High School Teacher and Community College Professor\n™ … Co-Author of: A Comedy of (Classroom) Errors … CEO and Founder of: Consiglio Devastations...… (more)\nProfile:\nJosh Fechter, Author | Advisor | Entrepreneur\nI’m the co-founder of a writing software, Squibler. … I’m also the founder of Product & Growth, a community of 25,000 members. … Want my latest book on scaling grow

In [40]:
# driver.quit()